In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [2]:
selectedmicrodata = pd.read_csv('/Users/francescamorini/projects/water-data/results/microdata/microdata_selectedrecipients.csv', sep=';')

In [3]:
selectedmicrodata.sample(5)

,Unnamed: 0,DonorName,RecipientName,IncomegroupName,USD_Commitment_Defl,USD_Received_Defl,ShortDescription,ProjectTitle,PurposeName,SectorName,ChannelName,ChannelReportedName,ExpectedStartDate,CompletionDate,LongDescription,CommitmentDate
342,224758,Germany,Tanzania,LDCs,16.014800,0.0,URBAN WATER SUPPLY II,URBAN WATER SUPPLY II,Water supply and sanitation - large systems,I.4. Water Supply & Sanitation,Public Sector Institutions,Public sector,2005-12-01,2005-12-01,NaN,2005-07-12
846,26521,Germany,Bolivia,LMICs,0.010903,-0.0,SAFTEY WATER,Saftey water,Basic drinking water supply,I.4. Water Supply & Sanitation,Donor country-based NGO,Donor country-based NGO,2011-01-01,2011-12-31,Saftey water,NaN
878,28866,Germany,Kenya,LMICs,0.028872,-0.0,WATER SUPPLY AND SANITATION -LARGE SYSTEMS,Water supply and sanitation -large systems,Water supply and sanitation - large systems,I.4. Water Supply & Sanitation,Public Sector Institutions,GIZ,2011-01-01,2011-12-31,support for the implementation of the German d...,NaN
461,21002,Germany,Tanzania,LDCs,0.323677,0.0,BASIC DRINKING WATER SUPPLY AND BASIC SANITATION,"Singa Village Water Supply, Tansania",Basic drinking water supply and basic sanitation,I.4. Water Supply & Sanitation,Non-Governmental Organisation (NGO) and Civil ...,NGOs and civil society,NaN,NaN,"Singa Village Water Supply, Tansania",NaN
376,266434,Germany,India,LMICs,0.027349,NaN,RAIL TRANSPORT,PLANING AND IMPLEMENTATION OF DECENTRALIZED WA...,Basic drinking water supply and basic sanitation,I.4. Water Supply & Sanitation,Non-Governmental Organisation (NGO) and Civil ...,NGOs and civil society,2005-05-15,2005-12-31,NaN,2005-12-31


Creating a slim version of data to avoid too many fields for vis and trailing commas problem

In [4]:
cleaned_selection = selectedmicrodata.drop(columns=["Unnamed: 0","ProjectTitle","PurposeName","SectorName","ExpectedStartDate","CompletionDate","LongDescription"])

In [5]:
cleaned_selection.sample(5)

,DonorName,RecipientName,IncomegroupName,USD_Commitment_Defl,USD_Received_Defl,ShortDescription,ChannelName,ChannelReportedName,CommitmentDate
1042,Germany,Ethiopia,LDCs,3.632030,-0.0,REGIONAL UNHCR PROJECT IN EAST AFRICA,United Nations Office of the United Nations Hi...,United Nations Office of the United Nations Hi...,2016-05-10
93,Germany,Burundi,LDCs,7.055230,NaN,RURAL WATER SUPPLY,NaN,MINISTERE DES FINANCES,1994-09-01
947,Germany,Burundi,LDCs,21.580900,-0.0,"WATER AND SANITATION SECTORAL PROGRAMME, PHASE...",Public Sector Institutions,KfW,2013-01-22
891,Germany,Uganda,LDCs,0.011421,-0.0,WATER RESOURCES POLICY AND ADMINISTRATIVE MANA...,Public Sector Institutions,GIZ,NaN
600,Germany,Sudan,LDCs,0.007898,-0.0,THE EXPERT IS ASSIGNED FOR TRAINING AND LECTUR...,Public Sector Institutions,Public sector,NaN


Exporting with standard sep and no index for prototyping

In [6]:
cleaned_selection.to_csv('selected_microdata.csv', sep=',', index=False)

Doing some calculations to get money % according to income groups

In [7]:
countries = cleaned_selection['RecipientName'].unique()

In [8]:
countries

array(['Tanzania', 'Bolivia', 'Kenya', 'Ethiopia', 'Sudan', 'Burundi',
       'Bangladesh', 'Lebanon', 'Egypt', 'Rwanda', 'India', 'Colombia',
       'Uganda'], dtype=object)

In [9]:
#Calculating projects % values for income groups and single countries

incomeNames = ['LMICs', 'LDCs', 'UMICs']
fullTotal = 0

for name in incomeNames: 
    total = cleaned_selection["USD_Commitment_Defl"].sum()
    sumElement = cleaned_selection["USD_Commitment_Defl"].loc[cleaned_selection["IncomegroupName"] == name].sum()
    subSelection = cleaned_selection.loc[cleaned_selection["IncomegroupName"] == name]
    perc = sumElement * 100 / total
    fullTotal = fullTotal + perc
    subTotal = 0
    print(name)
    print(perc)
    print('--------')
    
    for country in countries:
        sumCountry = subSelection["USD_Commitment_Defl"].loc[subSelection["RecipientName"] == country].sum() 
        subPerc = sumCountry * 100 / total
        subTotal = subTotal + subPerc
        if subPerc != 0:
            print(country)
            print(subPerc)
    print(subTotal)
    print('--------')
    
print(fullTotal)

LMICs
58.2412134936116
--------
Bolivia
11.95913699745467
Kenya
14.151360672533468
Egypt
17.260749453275313
India
14.86996637034815
58.2412134936116
--------
LDCs
34.88454453864079
--------
Tanzania
13.085009347279655
Ethiopia
2.121438322643363
Sudan
2.1670801576507426
Burundi
6.167040549987557
Bangladesh
2.5428288881536587
Rwanda
1.3280894132733618
Uganda
7.47305785965245
34.88454453864079
--------
UMICs
6.874241967747618
--------
Lebanon
4.828232301297417
Colombia
2.0460096664501997
6.874241967747617
--------
100.00000000000001


Working on time series to determine how spending evolved.

In [10]:
cleaned_selection["CommitmentDate"] = pd.to_datetime(cleaned_selection["CommitmentDate"])

In [11]:
cleaned_selection['year'] = cleaned_selection["CommitmentDate"].dt.year

In [12]:
cleaned_selection.sample(5)

,DonorName,RecipientName,IncomegroupName,USD_Commitment_Defl,USD_Received_Defl,ShortDescription,ChannelName,ChannelReportedName,CommitmentDate,year
364,Germany,Lebanon,UMICs,0.008995,NaN,WASTE MANAGEMENT/DISPOSAL,Public Sector Institutions,Public sector,2005-12-31,2005.0
256,Germany,Lebanon,UMICs,0.147231,NaN,DRINKING WATER STERILISATION,NaN,NaN,2002-12-31,2002.0
339,Germany,Tanzania,LDCs,8.559590,NaN,"RURAL WATER SUPPLY EAST KILIMAJA, PH. II",Public Sector Institutions,Public sector,2005-07-12,2005.0
885,Germany,Uganda,LDCs,0.003769,-0.0,WATER SUPPLY AND SANITATION -LARGE SYSTEMS,Public Sector Institutions,GIZ,NaT,NaN
1178,Germany,Kenya,LMICs,17.370500,0.0,WATER AND SANITATION TO PROTECT BIODIVERSITY,Central Government,Central Government,2019-12-30,2019.0


The data goes way back before 1980!

In [13]:
cleaned_selection["CommitmentDate"].min()

Timestamp('1973-05-01 00:00:00')

In [14]:
cleaned_selection["CommitmentDate"].max()

Timestamp('2019-12-31 00:00:00')

In [15]:
sorted_selection = cleaned_selection.sort_values(by='CommitmentDate', ascending=True, inplace=False, kind='quicksort', na_position='last')

In [17]:
cleaned_selection.to_csv('data/historically_ordered.csv', sep=',', index=False,na_rep='NaN')

Checking what's NaN. A lot of information on projects years and Donor's channel.

In [18]:
cleaned_selection.isna().sum()

DonorName                0
RecipientName            0
IncomegroupName          0
USD_Commitment_Defl      0
USD_Received_Defl      301
ShortDescription         0
ChannelName            308
ChannelReportedName    233
CommitmentDate         445
year                   445
dtype: int64

Generating a file with only labels and counts to see if there are recurring projects or similar projects appearing.

In [36]:
labels = cleaned_selection['ShortDescription'].unique()
value_counts = cleaned_selection['ShortDescription'].value_counts()


project_descriptions = pd.DataFrame(value_counts)

In [37]:
project_descriptions

,ShortDescription
BASIC DRINKING WATER SUPPLY AND BASIC SANITATION,94
WATER SUPPLY AND SANITATION - LARGE SYSTEMS,35
WATER RESOURCES PROTECTION,35
SCIENTIFIC COOPERATION WITH OTHER COUNTRIES (NOT INCLUDED IN OTHER AREAS),32
WATER RESOURCES POLICY/ADMIN. MGMT,27
...,...
WATER TANKS FOR TANZANIA,1
WATER FOR THE NOMADS IN NORTHERN KENYA,1
NAIROBI WATER DISTRIBUTION PROGRAMME,1
"WATER SUPPLY, TABORA",1


In [45]:
project_descriptions.reset_index(level=0, inplace=True)

In [50]:
clean_descriptions = project_descriptions.drop(columns=['index', 'level_0'])

In [51]:
clean_descriptions.to_csv('data/projects_descriptions.csv', sep=',', index=False,na_rep='NaN')

In [56]:
years = sorted_selection['year'].unique()

In [66]:
sumValues = []

for year in years:
    singleyear = sorted_selection.loc[sorted_selection['year'] == year, 'USD_Commitment_Defl'].sum()
    sumValues.append(singleyear)

import statistics
statistics.mean(sumValues)

89.10677331041667